In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/llm-detect-ai-generated-text/sample_submission.csv
/kaggle/input/llm-detect-ai-generated-text/train_prompts.csv
/kaggle/input/llm-detect-ai-generated-text/test_essays.csv
/kaggle/input/llm-detect-ai-generated-text/train_essays.csv


In [2]:
train = pd.read_csv(
    "/kaggle/input/llm-detect-ai-generated-text/train_essays.csv"
)
test = pd.read_csv("/kaggle/input/llm-detect-ai-generated-text/test_essays.csv")
# could sample for class balance, specific prompts etc

In [3]:
frames=[train,test]

In [4]:
df=pd.concat(frames)

In [5]:
df.head()

,id,prompt_id,text,generated
0,0059830c,0,Cars. Cars have been around since they became ...,0.0
1,005db917,0,Transportation is a large necessity in most co...,0.0
2,008f63e3,0,"""America's love affair with it's vehicles seem...",0.0
3,00940276,0,How often do you ride in a car? Do you drive a...,0.0
4,00c39458,0,Cars are a wonderful thing. They are perhaps o...,0.0


In [6]:
df['prompt_id'].value_counts()

prompt_id
0    708
1    670
2      1
3      1
4      1
Name: count, dtype: int64

In [7]:
df=df.drop(['id','prompt_id'],axis=1)

In [8]:
df.head()

,text,generated
0,Cars. Cars have been around since they became ...,0.0
1,Transportation is a large necessity in most co...,0.0
2,"""America's love affair with it's vehicles seem...",0.0
3,How often do you ride in a car? Do you drive a...,0.0
4,Cars are a wonderful thing. They are perhaps o...,0.0


In [9]:
df=df.dropna()

In [10]:
df.shape

(1378, 2)

In [11]:
df['generated'] = df['generated'].astype("int")

In [12]:
df.head()

,text,generated
0,Cars. Cars have been around since they became ...,0
1,Transportation is a large necessity in most co...,0
2,"""America's love affair with it's vehicles seem...",0
3,How often do you ride in a car? Do you drive a...,0
4,Cars are a wonderful thing. They are perhaps o...,0


In [13]:
from sklearn.model_selection import train_test_split
from datasets import Dataset, DatasetDict



# Split the DataFrame into training and test sets
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

# Convert DataFrames to Datasets
data = DatasetDict({
    'train': Dataset.from_pandas(train_df, split='train'),
    'test': Dataset.from_pandas(test_df, split='test'),
})

# Print the dataset_dict
print(data)


DatasetDict({
    train: Dataset({
        features: ['text', 'generated', '__index_level_0__'],
        num_rows: 1102
    })
    test: Dataset({
        features: ['text', 'generated', '__index_level_0__'],
        num_rows: 276
    })
})


In [14]:
data

DatasetDict({
    train: Dataset({
        features: ['text', 'generated', '__index_level_0__'],
        num_rows: 1102
    })
    test: Dataset({
        features: ['text', 'generated', '__index_level_0__'],
        num_rows: 276
    })
})

In [16]:
for split in data.keys():
    data[split] = data[split].remove_columns('__index_level_0__')


In [17]:
data

DatasetDict({
    train: Dataset({
        features: ['text', 'generated'],
        num_rows: 1102
    })
    test: Dataset({
        features: ['text', 'generated'],
        num_rows: 276
    })
})

In [18]:
for split in data.keys():
    data[split] = data[split].rename_column('generated', 'label')


In [19]:
data

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 1102
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 276
    })
})

In [20]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [21]:
def preprocess_function(examples):

    return tokenizer(examples["text"], truncation=True)

In [22]:
tokenized_data = data.map(preprocess_function, batched=True)

  0%|          | 0/2 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [23]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [24]:
!pip install transformers datasets evaluate accelerate

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  Obtaining dependency information for evaluate from https://files.pythonhosted.org/packages/70/63/7644a1eb7b0297e585a6adec98ed9e575309bb973c33b394dae66bc35c69/evaluate-0.4.1-py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 242.9 kB/s eta 0:00:00a 0:00:01


In [25]:
import evaluate

accuracy = evaluate.load("accuracy")

In [26]:
import numpy as np


def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)


In [27]:
id2label = {0: "Not Ai generated", 1: "Ai generated"}
label2id = {"Not Ai generated": 0, "Ai generated": 1}


In [28]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

model = AutoModelForSequenceClassification.from_pretrained(
    "distilbert-base-uncased", num_labels=2, id2label=id2label, label2id=label2id
)

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.weight', 'pre_classifier.bias', 'classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [29]:
training_args = TrainingArguments(
    output_dir="wastee",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=2,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_data["train"],
    eval_dataset=tokenized_data["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.023767,0.996377
2,No log,0.023855,0.996377


TrainOutput(global_step=138, training_loss=0.04585146903991699, metrics={'train_runtime': 3601.2717, 'train_samples_per_second': 0.612, 'train_steps_per_second': 0.038, 'total_flos': 291958146637824.0, 'train_loss': 0.04585146903991699, 'epoch': 2.0})

In [49]:
trainer.evaluate(eval_dataset=tokenized_data["test"])

{'eval_loss': 0.023766981437802315,
 'eval_accuracy': 0.9963768115942029,
 'eval_runtime': 120.948,
 'eval_samples_per_second': 2.282,
 'eval_steps_per_second': 0.149,
 'epoch': 2.0}